## The Code Explained
Let's break the code down.

In [3]:
import rospy
from std_msgs.msg import String

You need to import rospy if you are writing a ROS Node. The `std_msgs.msg` import is so that we can reuse the `std_msgs/String` message type (a simple string container) for publishing. 

In [4]:
pub = rospy.Publisher('chatter', String, queue_size=10)
rospy.init_node('talker', anonymous=True)

This section of code defines the talker's interface to the rest of ROS.  
`pub = rospy.Publisher("chatter", String, queue_size=10)` declares that your node is publishing to the `chatter` topic using the message type `String`. `String` here is actually the class `std_msgs.msg.String`. The `queue_size` argument is <span style="background-color:yellow">**New in ROS hydro**</span> and limits the amount of queued messages if any subscriber is not receiving them fast enough. In older ROS distributions just omit the argument.

The next line, `rospy.init_node(NAME, ...)`, is very important as it tells rospy the name of your node -- until rospy has this information, it cannot start communicating with the ROS [Master](http://wiki.ros.org/Master). In this case, your node will take on the name `talker`. NOTE: the name must be a [base name](http://wiki.ros.org/Names), i.e. it cannot contain any slashes "/".

`anonymous = True` ensures that your node has a unique name by adding random numbers to the end of NAME. [Refer to Initialization and Shutdown - Initializing your ROS Node](http://wiki.ros.org/rospy/Overview/Initialization%20and%20Shutdown#Initializing_your_ROS_Node) in the `rospy` documentation for more information about node initialization options. 

In [5]:
rate = rospy.Rate(10) # 10hz

This line creates a `Rate` object rate. With the help of its method `sleep()`, it offers a convenient way for looping at the desired rate. With its argument of 10, we should expect to go through the loop 10 times per second (as long as our processing time does not exceed 1/10th of a second!) 

In [6]:
def talker():
    while not rospy.is_shutdown():
        hello_str = "hello world %s" % rospy.get_time()
        rospy.loginfo(hello_str)
        pub.publish(hello_str)
        rate.sleep()

This loop is a fairly standard rospy construct: checking the `rospy.is_shutdown()` flag and then doing work. You have to check `is_shutdown()` to check if your program should exit (e.g. if there is a `Ctrl-C` or otherwise). In this case, the "work" is a call  to `pub.publish(hello_str)` that publishes a string to our chatter topic. The loop calls `rate.sleep()`, which sleeps just long enough to maintain the desired rate through the loop.

(You may also run across `rospy.sleep()` which is similar to `time.sleep()` except that it works with simulated time as well (see [Clock](http://wiki.ros.org/Clock)).) 

This loop also calls `rospy.loginfo(str)`, which performs triple-duty: the messages get printed to screen, it gets written to the Node's log file, and it gets written to [rosout](http://wiki.ros.org/rosout). [rosout](http://wiki.ros.org/rosout) is a handy for debugging: you can pull up messages using [rqt_console](http://wiki.ros.org/rqt_console) instead of having to find the console window with your Node's output. 

`std_msgs.msg.String` is a very simple message type, so you may be wondering what it looks like to publish more complicated types. The general rule of thumb is that constructor args are in the same order as in the *.msg* file. You can also pass in no arguments and initialize the fields directly, e.g. 

In [7]:
msg = String()
msg.data = str

or you can initialize some of the fields and leave the rest with default values: 

In [8]:
String(data=str)

data: <type 'str'>

You may be wondering about the last little bit: 

In [9]:
try:
    talker()
except rospy.ROSInterruptException:
    pass

[INFO] [1597196793.107699]: hello world 1597196793.11
[INFO] [1597196793.108265]: hello world 1597196793.11
[INFO] [1597196793.208880]: hello world 1597196793.21
[INFO] [1597196793.309027]: hello world 1597196793.31
[INFO] [1597196793.409000]: hello world 1597196793.41
[INFO] [1597196793.508688]: hello world 1597196793.51
[INFO] [1597196793.608979]: hello world 1597196793.61
[INFO] [1597196793.708704]: hello world 1597196793.71
[INFO] [1597196793.808984]: hello world 1597196793.81
[INFO] [1597196793.909671]: hello world 1597196793.91
[INFO] [1597196794.008598]: hello world 1597196794.01
[INFO] [1597196794.108651]: hello world 1597196794.11
[INFO] [1597196794.208973]: hello world 1597196794.21
[INFO] [1597196794.308587]: hello world 1597196794.31
[INFO] [1597196794.408966]: hello world 1597196794.41
[INFO] [1597196794.508746]: hello world 1597196794.51
[INFO] [1597196794.608687]: hello world 1597196794.61
[INFO] [1597196794.708973]: hello world 1597196794.71
[INFO] [1597196794.809009]: 

[INFO] [1597196808.208995]: hello world 1597196808.21
[INFO] [1597196808.308603]: hello world 1597196808.31
[INFO] [1597196808.408657]: hello world 1597196808.41
[INFO] [1597196808.509004]: hello world 1597196808.51
[INFO] [1597196808.608585]: hello world 1597196808.61
[INFO] [1597196808.708652]: hello world 1597196808.71
[INFO] [1597196808.808994]: hello world 1597196808.81
[INFO] [1597196808.908868]: hello world 1597196808.91
[INFO] [1597196809.008703]: hello world 1597196809.01
[INFO] [1597196809.108813]: hello world 1597196809.11
[INFO] [1597196809.208908]: hello world 1597196809.21
[INFO] [1597196809.308817]: hello world 1597196809.31
[INFO] [1597196809.408959]: hello world 1597196809.41
[INFO] [1597196809.509001]: hello world 1597196809.51
[INFO] [1597196809.608991]: hello world 1597196809.61
[INFO] [1597196809.708998]: hello world 1597196809.71
[INFO] [1597196809.808943]: hello world 1597196809.81
[INFO] [1597196809.909004]: hello world 1597196809.91
[INFO] [1597196810.009010]: 

KeyboardInterrupt: 

This catches a `rospy.ROSInterruptException` exception, which can be thrown by `rospy.sleep()` and `rospy.Rate.sleep()` methods when `Ctrl-C` is pressed or your Node is otherwise shutdown. The reason this exception is raised is so that you don't accidentally continue executing code after the `sleep()`. 